## Analisador Léxico

Analisador léxico para uma linguagem C-like que é capaz de identificar e classificar os seguintes elementos:
- **Palavras-chave**: int, float, for, if, etc.
- **Identificadores**: Nomes de variáveis e funções.
- **Literais**: Inteiros, flutuantes, strings, caracteres.
- **Operadores**: Aritméticos, lógicos, de atribuição, etc.
- **Delimitadores**: Parênteses, chaves, ponto e vírgula.
- **Comentários**: Comentários de linha e de bloco.

In [25]:
import re
from tabulate import tabulate
from typing import List, Tuple, Dict, Any

### 1. Implementação do Analisador Léxico

In [26]:
class Lexer:
    """
    Analisador Léxico para uma linguagem C-like.
    
    Responsável por converter um código fonte em uma sequência de tokens,
    identificando palavras-chave, identificadores, literais, operadores e erros.
    """

    # --- Configuração Estática (Melhora a Modularidade) ---
    _KEYWORDS = {
        "int", "float", "double", "char", "void", "if", "else", "while", "for", "return",
        "switch", "case", "default", "break", "continue", "struct", "union", "enum",
        "typedef", "const", "static", "extern", "goto", "sizeof"
    }

    _OPERATORS_MAP = {
        "ARITH_OP": {"+", "-", "*", "/", "%"},
        "REL_OP": {"==", "!=", "<", "<=", ">", ">="},
        "LOGIC_OP": {"&&", "||", "!"},
        "BIT_OP": {"&", "|", "^", "~", "<<", ">>"},
        "ASSIGN_OP": {"=", "+=", "-=", "*=", "/=", "%=", "&=", "|=", "^=", "<<=", ">>="},
        "INCDEC": {"++", "--"},
        "ARROW": {"->"},
        "ELLIPSIS": {"..."},
        "TERNARY": {"?", ":"},
    }

    _DELIMITERS_MAP = {
        "LPAREN": {"("}, "RPAREN": {")"},
        "LBRACE": {"{"}, "RBRACE": {"}"},
        "LBRACKET": {"["}, "RBRACKET": {"]"},
        "SEMICOLON": {";"},
        "COMMA": {","},
        "DOT": {"."},
    }

    # --- Especificação dos Tokens (Ordem é Importante) ---
    _TOKEN_SPECIFICATION = [
        ("PP_DIRECTIVE", r"^[ \t]*\#.*"),
        ("COMMENT_BLOCK", r"/\*[\s\S]*?\*/"),
        ("COMMENT_LINE", r"//.*"),
        ("STRING", r'"(?:\\.|[^"\\])*"'),
        ("CHAR", r"'(?:\\.|[^'\\])?'"),
        ("ERROR_CHAR_MULTI", r"'.{2,}'"),
        ("ERROR_UNTERM_STRING", r'"(?:[^"/\n]|/(?![/*]))*'),
        ("ERROR_UNTERM_CHAR", r"'(?:[^'/\n]|/(?![/*]))*"),
        ("ERROR_NUM_ID", r"\d+[A-Za-z_][A-Za-z0-9_]*"),
        ("ERROR_NUMBER_COMMA", r"\d+,\d+"),
        ("ELLIPSIS", r"\.\.\."),
        ("OP_3", r"<<=|>>="),
        ("OP_2", r"==|!=|<=|>=|\+=|-=|\*=|/=|%=|&=|\|=|\^=|<<|>>|&&|\|\||\+\+|--|->"),
        ("OP_1", r"[+\-*/%<>=!&|^~?:]"),
        ("DELIM", r"[()\[\]{};,\.]"),
        ("FLOAT", r"\d+\.\d+"),
        ("INT", r"\d+"),
        ("ID", r"[A-Za-z_][A-Za-z0-9_]*"),
        ("NEWLINE", r"\n+"),
        ("SKIP", r"[ \t]+"),
        ("ERROR_MISMATCH", r"."),
    ]

    def __init__(self, source: str):
        self.src = source
        self.tokens: List[Tuple[str, str, Any, int, int]] = []
        self.symbols: Dict[str, Dict[str, Any]] = {}
        self.next_id = 1

        # Compila o regex uma vez para otimizar a performance
        self.tok_regex = re.compile(
            "|".join(f"(?P<{pair[0]}>{pair[1]})" for pair in self._TOKEN_SPECIFICATION),
            re.MULTILINE
        )

        # Mapeamento de operadores para suas categorias (para performance)
        self._flat_op_to_cat = self._build_flat_op_map()

        # Mapeamento de tipos de erro para mensagens (centraliza as mensagens)
        self._error_messages = {
            'ERROR_UNTERM_STRING': "string não terminada",
            'ERROR_UNTERM_CHAR': "literal de caractere não terminado",
            'ERROR_CHAR_MULTI': "literal de caractere com múltiplos caracteres",
            'ERROR_NUM_ID': "identificador não pode começar com número",
            'ERROR_NUMBER_COMMA': "vírgula como separador decimal (use ponto)",
            'ERROR_MISMATCH': "caractere inesperado",
        }

    @classmethod
    def _build_flat_op_map(cls) -> Dict[str, str]:
        """Cria um dicionário para categorizar operadores e delimitadores rapidamente."""
        flat_map = {}
        for cat, op_set in {**cls._OPERATORS_MAP, **cls._DELIMITERS_MAP}.items():
            for op in op_set:
                flat_map[op] = cat
        return flat_map

    def tokenize(self) -> Tuple[List[Any], Dict[str, Any]]:
        """
        Executa a análise léxica no código fonte.

        Itera sobre todas as correspondências encontradas pelo regex compilado,
        ignora o que não é relevante (comentários, espaços) e despacha
        cada token para o método de tratamento apropriado.
        """
        for mo in self.tok_regex.finditer(self.src):
            kind = mo.lastgroup
            
            if kind in ("COMMENT_BLOCK", "COMMENT_LINE", "SKIP", "NEWLINE"):
                continue

            lexeme = mo.group()
            line, col = self._get_line_col(mo.start())
            
            # --- Despacho de Handlers (Melhora a Legibilidade) ---
            if kind.startswith("ERROR"):
                self._handle_error(kind, lexeme, line, col)
            elif kind in ("STRING", "CHAR"):
                self._handle_literal(kind, lexeme, line, col)
            elif kind in ("FLOAT", "INT"):
                self._handle_numeric_literal(kind, lexeme, line, col)
            elif kind == "ID":
                self._handle_identifier(lexeme, line, col)
            elif kind == "PP_DIRECTIVE":
                self._handle_pp_directive(lexeme, line, col)
            else: # Operadores e Delimitadores
                self._handle_operator(lexeme, line, col)

        # Adiciona token de Fim de Arquivo (EOF)
        eof_line, eof_col = self._get_line_col(len(self.src))
        self._add_token("EOF", "", None, eof_line, eof_col)
        
        return self.tokens, self.symbols

    # --- Métodos de Tratamento (Handlers) ---

    def _add_token(self, ttype: str, lexeme: str, attr: Any, line: int, col: int):
        """Adiciona um token formatado à lista de tokens."""
        self.tokens.append((ttype, lexeme, attr, line, col))

    def _handle_error(self, kind: str, lexeme: str, line: int, col: int):
        message = self._error_messages.get(kind, "Erro desconhecido")
        self._add_token("ERROR", lexeme, message, line, col)

    def _handle_literal(self, kind: str, lexeme: str, line: int, col: int):
        ttype = "STRING_LITERAL" if kind == "STRING" else "CHAR_LITERAL"
        content = lexeme[1:-1]
        self._add_token(ttype, lexeme, content, line, col)

    def _handle_numeric_literal(self, kind: str, lexeme: str, line: int, col: int):
        try:
            if kind == "FLOAT":
                val = float(lexeme)
                self._add_token("FLOAT_LITERAL", lexeme, val, line, col)
            else: # INT
                val = int(lexeme)
                self._add_token("INT_LITERAL", lexeme, val, line, col)
        except ValueError:
            msg = "float inválido" if kind == "FLOAT" else "inteiro inválido"
            self._add_token("ERROR", lexeme, msg, line, col)
    
    def _handle_identifier(self, lexeme: str, line: int, col: int):
        if lexeme in self._KEYWORDS:
            self._add_token("KEYWORD", lexeme, None, line, col)
        else:
            assigned_id = self._add_symbol(lexeme)
            self._add_token("IDENTIFIER", lexeme, assigned_id, line, col)

    def _handle_pp_directive(self, lexeme: str, line: int, col: int):
        self._add_token("PP_DIRECTIVE", lexeme.rstrip("\r\n"), None, line, col)

    def _handle_operator(self, lexeme: str, line: int, col: int):
        cat = self._flat_op_to_cat.get(lexeme, "UNKNOWN_OP_DELIM")
        self._add_token(cat, lexeme, None, line, col)

    # --- Métodos Auxiliares ---

    def _get_line_col(self, pos: int) -> Tuple[int, int]:
        """Calcula a linha e coluna (1-based) para uma dada posição no código."""
        line = self.src.count('\n', 0, pos) + 1
        last_nl = self.src.rfind('\n', 0, pos)
        col = (pos - last_nl) if last_nl != -1 else pos + 1
        return line, col

    def _add_symbol(self, name: str) -> str:
        """Adiciona um novo identificador à tabela de símbolos ou incrementa a contagem."""
        if name not in self.symbols:
            assigned_id = f"id{self.next_id}"
            self.symbols[name] = {"id": assigned_id, "count": 1}
            self.next_id += 1
        else:
            self.symbols[name]["count"] += 1
        return self.symbols[name]["id"]

    def pretty_print(self):
        """Imprime as tabelas de tokens e de símbolos de forma legível."""
        # Tabela de Tokens
        token_rows = [
            [f"{line}:{col}", ttype, lex, attr if attr is not None else ""]
            for ttype, lex, attr, line, col in self.tokens
        ]
        print("\nTabela de Tokens:")
        print(tabulate(token_rows, headers=["Pos", "Tipo", "Lexema", "Atributo"], tablefmt="fancy_grid"))

        # Tabela de Símbolos
        def id_key(item):
            return int(item[1]["id"][2:]) if item[1]["id"].startswith("id") else 0
        
        sorted_symbols = sorted(self.symbols.items(), key=id_key)
        sym_rows = [
            [data["id"], name, data["count"]]
            for name, data in sorted_symbols
        ]
        print("\nTabela de Símbolos:")
        print(tabulate(sym_rows, headers=["ID", "Identificador", "Ocorrências"], tablefmt="fancy_grid"))

### 2. Auxiliar da Análise Léxica

In [27]:
def analyze_source_code(source_code: str):
    """Função auxiliar para instanciar, tokenizar e imprimir os resultados."""
    lexer = Lexer(source_code)
    lexer.tokenize()
    lexer.pretty_print()

### 3. Testes Válidos

In [28]:
# 1. Operações bitwise e ponteiros (->, <<, &)
test_valid_1 = r'''
int main() {
    struct_ptr->value = (mask & 0xFF) << 2;
    return 0;
}
'''

# Execução dos Testes
analyze_source_code(test_valid_1)


Tabela de Tokens:
╒═══════╤═════════════╤════════════╤═══════════════════════════════════════════╕
│ Pos   │ Tipo        │ Lexema     │ Atributo                                  │
╞═══════╪═════════════╪════════════╪═══════════════════════════════════════════╡
│ 2:1   │ KEYWORD     │ int        │                                           │
├───────┼─────────────┼────────────┼───────────────────────────────────────────┤
│ 2:5   │ IDENTIFIER  │ main       │ id1                                       │
├───────┼─────────────┼────────────┼───────────────────────────────────────────┤
│ 2:9   │ LPAREN      │ (          │                                           │
├───────┼─────────────┼────────────┼───────────────────────────────────────────┤
│ 2:10  │ RPAREN      │ )          │                                           │
├───────┼─────────────┼────────────┼───────────────────────────────────────────┤
│ 2:12  │ LBRACE      │ {          │                                           │
├───────┼

In [29]:
# 2. Notação científica e atribuição composta
test_valid_2 = r'''
void calc() {
    float tolerance = 1.5e-4; // O lexer identifica como FLOAT (d+.d+) e ID/OP
    x += 42.0;
}
'''

# Execução dos Testes
analyze_source_code(test_valid_2)


Tabela de Tokens:
╒═══════╤═══════════════╤═══════════╤════════════╕
│ Pos   │ Tipo          │ Lexema    │ Atributo   │
╞═══════╪═══════════════╪═══════════╪════════════╡
│ 2:1   │ KEYWORD       │ void      │            │
├───────┼───────────────┼───────────┼────────────┤
│ 2:6   │ IDENTIFIER    │ calc      │ id1        │
├───────┼───────────────┼───────────┼────────────┤
│ 2:10  │ LPAREN        │ (         │            │
├───────┼───────────────┼───────────┼────────────┤
│ 2:11  │ RPAREN        │ )         │            │
├───────┼───────────────┼───────────┼────────────┤
│ 2:13  │ LBRACE        │ {         │            │
├───────┼───────────────┼───────────┼────────────┤
│ 3:5   │ KEYWORD       │ float     │            │
├───────┼───────────────┼───────────┼────────────┤
│ 3:11  │ IDENTIFIER    │ tolerance │ id2        │
├───────┼───────────────┼───────────┼────────────┤
│ 3:21  │ ASSIGN_OP     │ =         │            │
├───────┼───────────────┼───────────┼────────────┤
│ 3:23  │ FL

In [30]:
# 3. Strings com caracteres de escape e comentários mistos
test_valid_3 = r'''
char* msg = "Linha 1\nLinha 2\"Citação\""; // String complexa
/* Comentário de bloco 
   que abrange linhas */
// Comentário de linha
'''

# Execução dos Testes
analyze_source_code(test_valid_3)


Tabela de Tokens:
╒═══════╤════════════════╤═══════════════════════════════╤═════════════════════════════╕
│ Pos   │ Tipo           │ Lexema                        │ Atributo                    │
╞═══════╪════════════════╪═══════════════════════════════╪═════════════════════════════╡
│ 2:1   │ KEYWORD        │ char                          │                             │
├───────┼────────────────┼───────────────────────────────┼─────────────────────────────┤
│ 2:5   │ ARITH_OP       │ *                             │                             │
├───────┼────────────────┼───────────────────────────────┼─────────────────────────────┤
│ 2:7   │ IDENTIFIER     │ msg                           │ id1                         │
├───────┼────────────────┼───────────────────────────────┼─────────────────────────────┤
│ 2:11  │ ASSIGN_OP      │ =                             │                             │
├───────┼────────────────┼───────────────────────────────┼─────────────────────────────┤
│ 

In [31]:
# 4. Estruturas de controle aninhadas e operadores lógicos
test_valid_4 = r'''
while (a >= 10 && (b != 0 || !c)) {
    do_something();
}
'''

# Execução dos Testes
analyze_source_code(test_valid_4)


Tabela de Tokens:
╒═══════╤═════════════╤══════════════╤════════════╕
│ Pos   │ Tipo        │ Lexema       │ Atributo   │
╞═══════╪═════════════╪══════════════╪════════════╡
│ 2:1   │ KEYWORD     │ while        │            │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:7   │ LPAREN      │ (            │            │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:8   │ IDENTIFIER  │ a            │ id1        │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:10  │ REL_OP      │ >=           │            │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:13  │ INT_LITERAL │ 10           │ 10         │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:16  │ LOGIC_OP    │ &&           │            │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:19  │ LPAREN      │ (            │            │
├───────┼─────────────┼──────────────┼────────────┤
│ 2:20  │ IDENTIFIER  │ b            │ id2        │
├───────┼─────────────┼──────────────┼───────

In [32]:
# 5. Diretivas de pré-processador e caracteres
test_valid_5 = r'''
#define MAX_SIZE 100
#include <math.h>
char c = 'z';
'''

# Execução dos Testes
analyze_source_code(test_valid_5)


Tabela de Tokens:
╒═══════╤══════════════╤══════════════════════╤════════════╕
│ Pos   │ Tipo         │ Lexema               │ Atributo   │
╞═══════╪══════════════╪══════════════════════╪════════════╡
│ 2:1   │ PP_DIRECTIVE │ #define MAX_SIZE 100 │            │
├───────┼──────────────┼──────────────────────┼────────────┤
│ 3:1   │ PP_DIRECTIVE │ #include <math.h>    │            │
├───────┼──────────────┼──────────────────────┼────────────┤
│ 4:1   │ KEYWORD      │ char                 │            │
├───────┼──────────────┼──────────────────────┼────────────┤
│ 4:6   │ IDENTIFIER   │ c                    │ id1        │
├───────┼──────────────┼──────────────────────┼────────────┤
│ 4:8   │ ASSIGN_OP    │ =                    │            │
├───────┼──────────────┼──────────────────────┼────────────┤
│ 4:10  │ CHAR_LITERAL │ 'z'                  │ z          │
├───────┼──────────────┼──────────────────────┼────────────┤
│ 4:13  │ SEMICOLON    │ ;                    │            │
├────

### 4. Testes Inválidos

In [33]:
# 1. Identificador começando com número (ERROR_NUM_ID)
test_invalid_1 = r'''
int 123variavel = 10;
'''

# Execução dos Testes
analyze_source_code(test_invalid_1)


Tabela de Tokens:
╒═══════╤═════════════╤═════════════╤═══════════════════════════════════════════╕
│ Pos   │ Tipo        │ Lexema      │ Atributo                                  │
╞═══════╪═════════════╪═════════════╪═══════════════════════════════════════════╡
│ 2:1   │ KEYWORD     │ int         │                                           │
├───────┼─────────────┼─────────────┼───────────────────────────────────────────┤
│ 2:5   │ ERROR       │ 123variavel │ identificador não pode começar com número │
├───────┼─────────────┼─────────────┼───────────────────────────────────────────┤
│ 2:17  │ ASSIGN_OP   │ =           │                                           │
├───────┼─────────────┼─────────────┼───────────────────────────────────────────┤
│ 2:19  │ INT_LITERAL │ 10          │ 10                                        │
├───────┼─────────────┼─────────────┼───────────────────────────────────────────┤
│ 2:21  │ SEMICOLON   │ ;           │                                          

In [34]:
# 2. String não terminada (ERROR_UNTERM_STRING)
test_invalid_2 = r'''
char* texto = "Esta string nunca fecha;
int x = 10;
'''

# Execução dos Testes
analyze_source_code(test_invalid_2)


Tabela de Tokens:
╒═══════╤═════════════╤═══════════════════════════╤══════════════════════╕
│ Pos   │ Tipo        │ Lexema                    │ Atributo             │
╞═══════╪═════════════╪═══════════════════════════╪══════════════════════╡
│ 2:1   │ KEYWORD     │ char                      │                      │
├───────┼─────────────┼───────────────────────────┼──────────────────────┤
│ 2:5   │ ARITH_OP    │ *                         │                      │
├───────┼─────────────┼───────────────────────────┼──────────────────────┤
│ 2:7   │ IDENTIFIER  │ texto                     │ id1                  │
├───────┼─────────────┼───────────────────────────┼──────────────────────┤
│ 2:13  │ ASSIGN_OP   │ =                         │                      │
├───────┼─────────────┼───────────────────────────┼──────────────────────┤
│ 2:15  │ ERROR       │ "Esta string nunca fecha; │ string não terminada │
├───────┼─────────────┼───────────────────────────┼──────────────────────┤
│ 3:1 

In [35]:
# 3. Uso de vírgula em float (ERROR_NUMBER_COMMA)
test_invalid_3 = r'''
float pi_errado = 3,1415;
'''

# Execução dos Testes
analyze_source_code(test_invalid_3)


Tabela de Tokens:
╒═══════╤════════════╤═══════════╤════════════════════════════════════════════╕
│ Pos   │ Tipo       │ Lexema    │ Atributo                                   │
╞═══════╪════════════╪═══════════╪════════════════════════════════════════════╡
│ 2:1   │ KEYWORD    │ float     │                                            │
├───────┼────────────┼───────────┼────────────────────────────────────────────┤
│ 2:7   │ IDENTIFIER │ pi_errado │ id1                                        │
├───────┼────────────┼───────────┼────────────────────────────────────────────┤
│ 2:17  │ ASSIGN_OP  │ =         │                                            │
├───────┼────────────┼───────────┼────────────────────────────────────────────┤
│ 2:19  │ ERROR      │ 3,1415    │ vírgula como separador decimal (use ponto) │
├───────┼────────────┼───────────┼────────────────────────────────────────────┤
│ 2:25  │ SEMICOLON  │ ;         │                                            │
├───────┼────────────

In [36]:
# 4. Literal de caractere com múltiplos caracteres (ERROR_CHAR_MULTI)
test_invalid_4 = r'''
char c = 'abc'; 
'''

# Execução dos Testes
analyze_source_code(test_invalid_4)


Tabela de Tokens:
╒═══════╤════════════╤══════════╤═══════════════════════════════════════════════╕
│ Pos   │ Tipo       │ Lexema   │ Atributo                                      │
╞═══════╪════════════╪══════════╪═══════════════════════════════════════════════╡
│ 2:1   │ KEYWORD    │ char     │                                               │
├───────┼────────────┼──────────┼───────────────────────────────────────────────┤
│ 2:6   │ IDENTIFIER │ c        │ id1                                           │
├───────┼────────────┼──────────┼───────────────────────────────────────────────┤
│ 2:8   │ ASSIGN_OP  │ =        │                                               │
├───────┼────────────┼──────────┼───────────────────────────────────────────────┤
│ 2:10  │ ERROR      │ 'abc'    │ literal de caractere com múltiplos caracteres │
├───────┼────────────┼──────────┼───────────────────────────────────────────────┤
│ 2:15  │ SEMICOLON  │ ;        │                                              

In [37]:
# 5. Literal de caractere não terminado (ERROR_UNTERM_CHAR)
test_invalid_5 = r'''
char c = 'a;
return 0;
'''

# Execução dos Testes
analyze_source_code(test_invalid_5)


Tabela de Tokens:
╒═══════╤═════════════╤══════════╤════════════════════════════════════╕
│ Pos   │ Tipo        │ Lexema   │ Atributo                           │
╞═══════╪═════════════╪══════════╪════════════════════════════════════╡
│ 2:1   │ KEYWORD     │ char     │                                    │
├───────┼─────────────┼──────────┼────────────────────────────────────┤
│ 2:6   │ IDENTIFIER  │ c        │ id1                                │
├───────┼─────────────┼──────────┼────────────────────────────────────┤
│ 2:8   │ ASSIGN_OP   │ =        │                                    │
├───────┼─────────────┼──────────┼────────────────────────────────────┤
│ 2:10  │ ERROR       │ 'a;      │ literal de caractere não terminado │
├───────┼─────────────┼──────────┼────────────────────────────────────┤
│ 3:1   │ KEYWORD     │ return   │                                    │
├───────┼─────────────┼──────────┼────────────────────────────────────┤
│ 3:8   │ INT_LITERAL │ 0        │ 0         

### 5. Teste Extra (com erros)

In [38]:
source_code_4 = r'''
#include <stdio.h>
#define PI 3.14

int main() {
    // Casos válidos
    char c = '\n';
    char empty_c = '';    // char vazio (agora válido)
    char s_vazia[] = "";  // string vazia (sempre foi válida)
    float x = 3.14;
    int y = 42;
    y += 5;
    if (x >= 2.0 && y != 0) {
        printf("valor: %d\n", y);
    }
    // erros propositais:
    float e1 = 3,14;      // vírgula no float
    int 123abc = 10;      // id começa com número
    char bad_char = 'ab'; // char com >1 caractere
    char unterm_c = 'a;    // char não terminado
    char s[] = "texto sem fechar; // string nao terminada
    a -> b;
    a ... b;
}
'''

analyze_source_code(source_code_4)


Tabela de Tokens:
╒═══════╤════════════════╤════════════════════╤═══════════════════════════════════════════════╕
│ Pos   │ Tipo           │ Lexema             │ Atributo                                      │
╞═══════╪════════════════╪════════════════════╪═══════════════════════════════════════════════╡
│ 2:1   │ PP_DIRECTIVE   │ #include <stdio.h> │                                               │
├───────┼────────────────┼────────────────────┼───────────────────────────────────────────────┤
│ 3:1   │ PP_DIRECTIVE   │ #define PI 3.14    │                                               │
├───────┼────────────────┼────────────────────┼───────────────────────────────────────────────┤
│ 5:1   │ KEYWORD        │ int                │                                               │
├───────┼────────────────┼────────────────────┼───────────────────────────────────────────────┤
│ 5:5   │ IDENTIFIER     │ main               │ id1                                           │
├───────┼────────────